In [ ]:
import torch
import torch.nn as nn
from diffusers.utils import load_image
# from diffusers import FluxControlNetInpaintPipeline #, FluxImg2ImgPipeline
from diffusers.models.controlnet_flux import FluxControlNetModel

from diffusers.utils import load_image, check_min_version
from transformers import pipeline ,SegformerImageProcessor, AutoModelForSemanticSegmentation
import os
import time
import cProfile
from pstats import Stats
# from optimum.quanto import qfloat8,qint4,qint8, quantize,freeze
from quanto import qfloat8,qint4,qint8, quantize,freeze

In [ ]:
controlnet_model = 'InstantX/FLUX.1-dev-Controlnet-Union'
controlnet = FluxControlNetModel.from_pretrained(controlnet_model, torch_dtype=torch.bfloat16,add_prefix_space=True,guidance_embeds=False)


In [ ]:
import glob
# from random import shuffle
# # dir=glob.glob('test-ai/orig/*')
# # dir+=glob.glob('test-ai/jcrew_orig/*')
# # dir=glob.glob('test-ai/paulo_orig/*')
# # dir=glob.glob('test-ai/upscaled/*')
dir=glob.glob('test-ai/more_paulo/*')

# shuffle(dir)

In [ ]:
# !pip install opencv-python

In [ ]:
from LookBuilderPipeline.resize import resize_images
from LookBuilderPipeline.segment import segment_image
from LookBuilderPipeline.pose import detect_pose
# from LookBuilderPipeline.plot_images import showImagesHorizontally

In [ ]:
def image_setup(input_image):
    image = load_image(input_image)
    pose_image = detect_pose(input_image)
    mask,mask_image,mask_array = segment_image(input_image,inverse=True,additional_option='shoe')
    mask2,mask_image2,mask_array2 = segment_image(input_image,inverse=False,additional_option='shoe')
            
    # if image.size[0] < image.size[0]:
    #             sm_image=resize_images(image,sm_pose_image.size,aspect_ratio=sm_pose_image.size[0]/sm_pose_image.size[1])
    #             sm_pose_image=resize_images(pose_image,sm_pose_image.size,aspect_ratio=None)
    #             sm_mask=resize_images(mask_image,sm_pose_image.size,aspect_ratio=sm_pose_image.size[0]/sm_pose_image.size[1])
        
    # else:
    sm_image=resize_images(image,target_size=image.size,aspect_ratio=None)
    sm_pose_image=resize_images(pose_image,image.size,aspect_ratio=image.size[0]/image.size[1])
    sm_mask=resize_images(mask_image,image.size,aspect_ratio=image.size[0]/image.size[1])
    return sm_image, sm_pose_image, sm_mask

In [ ]:
import numpy as np
def showImagesHorizontally(list_of_files, prompt,negative_prompt,model,time,height,width,controlnet_conditioning_scale,num_inference_steps, guidance_scale,seed,strength,output_path):
        from matplotlib.pyplot import figure, imshow, axis
        from matplotlib.image import imread
        import matplotlib.pyplot as plt

        fig = figure(figsize=(10,5))
        number_of_files = len(list_of_files)
        for i in range(number_of_files):
            a=fig.add_subplot(1,number_of_files,i+1)
            image = (list_of_files[i])
            imshow(image,cmap='Greys_r')
            axis('off')

        # Add text to the image
        fig.text(0.5, 0.01, f"Prompt: {prompt}       Neg_Prompt: {negative_prompt} \n Model: {model}  Time(s): {np.round(time,2)}  Time(m): {np.round(time/60,2)}  height: {height}  width: {width}  cond_scale: {controlnet_conditioning_scale}  steps: {num_inference_steps}  guidance: {guidance_scale} strength: {strength} seed: {seed}", ha='center', fontsize=10, color='black', wrap=True)
        text_to_save = f"""
        Prompt: {prompt} 
        Neg_Prompt: {negative_prompt}
        Model: {model}
        Time: {time}
        height: {height}
        width: {width}
        cond_scale: {controlnet_conditioning_scale}
        steps: {num_inference_steps}
        guidance: {guidance_scale}
        seed: {seed}
        strength:{strength}
        time: {time}"""
        
         # Save the text to a .txt file
        with open(output_path+'.txt', 'w') as file:  # Specify the desired file name
            file.write(text_to_save)  # Write the text to the file
        
        plt.tight_layout()  # Adjust the layout to prevent overlapping
        plt.savefig(output_path, dpi=300, bbox_inches='tight')  # Save the figure
        plt.close(fig)  # Close the figure to free up memory


In [ ]:
# [sm_image.size,sm_pose_image.size,sm_mask.size]

In [ ]:
# showImagesHorizontally([sm_image,sm_pose_image,sm_mask])

### run with 2 terminals: command `watch nvidia-smi` and `top`

#### DEV 1800-2500mb gpumem 60-85% Volatile GPU-Util

In [ ]:
# import os
# import inspect
# ## change flux inpainting pipeline to allow negative-prompt in OpenFlux
# # PTH=(os.path.abspath(inspect.getfile(FluxControlNetInpaintPipeline)))
# PTH2=(os.path.abspath(inspect.getfile(FluxImg2ImgPipeline)))
# import shutil
# shutil.copy('LookBuilderPipeline/LookBuilderPipeline/image_models/openflux/orig_pipeline_flux_controlnet_inpainting.py', PTH2)
# # PTH

In [ ]:
# pipe = FluxControlNetInpaintPipeline.from_pretrained("black-forest-labs/FLUX.1-dev", controlnet=controlnet, torch_dtype=torch.bfloat16)

# pipe.enable_model_cpu_offload()
# pipe.text_encoder.to(torch.bfloat16)
# pipe.enable_sequential_cpu_offload()

# prompt='a photo realistic image of  black fashion model in atlanta georgia'
# # negative_prompt="ugly, bad quality, bad anatomy, deformed body, deformed hands, deformed feet, deformed face, deformed clothing, deformed skin, bad skin, leggings, tights, stockings, pants, sleeves",

# pr = cProfile.Profile()
# pr.enable()
# start1 = time.time()

# image_res = pipe(
#                 prompt,
#                 # negative_prompt=negative_prompt,
#                 image=sm_image,
#                 control_image=sm_pose_image,
#                 control_mode=4,
#                 # padding_mask_crop=32,
#                 controlnet_conditioning_scale=controlnet_conditioning_scale,
#                 mask_image=sm_mask,
#                 height=height,
#                 width=width,
#                 strength=0.99,
#                 num_inference_steps=num_inference_steps,
#                 guidance_scale=guidance_scale,
#                 generator=generator,
#                 # prompt_embeds=embeds,
#                 # pooled_prompt_embeds=embeds,
#             ).images[0]
# end1 = time.time()
# with open('CN_benchmark/dev_test.txt', 'w') as stream:
#     stats = Stats(pr,stream=stream)
#     stats.sort_stats('tottime').print_stats(50)

# pr.disable()
# image_res.save('CN_benchmark/dev_test.png')
# image_res

In [ ]:
# pipe = FluxControlNetInpaintPipeline.from_pretrained("black-forest-labs/FLUX.1-dev", controlnet=controlnet, torch_dtype=torch.bfloat16)

# pipe.enable_model_cpu_offload()
# pipe.text_encoder.to(torch.bfloat16)
# pipe.enable_sequential_cpu_offload()
# prompts=['a photo realistic image of a fashion model on a beach',
#         'a photo realistic image of a fashion model on a sailboat',
#         'a photo realistic image of a fashion model in a temple']
# for input_image in dir:
#     for j,prompt in enumerate(prompts):
#         sm_image, sm_pose_image, sm_mask= image_setup(input_image)
#         if j==0:
#             jj='beach'
#         elif j==1:
#             jj='boat'
#         elif j==2:
#             jj='temple'
#         i=os.path.basename(input_image).split('.')[0]
#         width,height=sm_image.size
        
#         num_inference_steps=10
#         guidance_scale=7.5
#         controlnet_conditioning_scale=0.99
#         seed=420042
#         generator = torch.Generator(device="cuda").manual_seed(seed)
#         negative_prompt='ugly, bad quality, bad anatomy, deformed body, deformed hands, deformed feet, deformed face, deformed clothing, deformed skin, bad skin, leggings, tights, stockings, pants, sleeves'

#         pr = cProfile.Profile()
#         pr.enable()
#         start1 = time.time()


#         image_res = pipe(
#                 prompt,
#                 # negative_prompt=negative_prompt,
#                 image=sm_image,
#                 control_image=sm_pose_image,
#                 control_mode=4,
#                 padding_mask_crop=32,
#                 controlnet_conditioning_scale=controlnet_conditioning_scale,
#                 mask_image=sm_mask,
#                 height=height,
#                 width=width,
#                 strength=0.9,
#                 num_inference_steps=num_inference_steps,
#                 guidance_scale=guidance_scale,
#                 generator=generator,
#                 # prompt_embeds=embeds,
#                 # pooled_prompt_embeds=embeds,
#             ).images[0]
#         end1 = time.time()
#         with open('dev_benchmark/dev_test'+str(i)+'_'+str(jj)+'.txt', 'w') as stream:
#             stats = Stats(pr,stream=stream)
#             stats.sort_stats('tottime').print_stats(50)
        
#         pr.disable()
#         # image_res.save('dev_benchmark/dev_test.png')
#         # image_res

        
#         # image_res.save('dev_benchmark/dev_test_'+str(i)+'_'+str(jj)+'.png')
#         filename='dev_benchmark/dev_bench_'+str(i)+'_'+str(jj)+'.png'
#         showImagesHorizontally([sm_image,sm_mask,sm_pose_image,image_res], prompt,negative_prompt,'dev',height, width, controlnet_conditioning_scale,num_inference_steps,guidance_scale,generator,output_path=filename)


#### SCHENLL 2000-2500mb gpumem 55-80% Volatile GPU-Util

In [ ]:
# pipe = FluxControlNetInpaintPipeline.from_pretrained("black-forest-labs/FLUX.1-schnell", controlnet=controlnet, torch_dtype=torch.bfloat16)

# pipe.enable_model_cpu_offload()
# pipe.text_encoder.to(torch.bfloat16)
# pipe.enable_sequential_cpu_offload()

# prompt='a photo realistic image of  black fashion model in atlanta georgia'
# negative_prompt="ugly, bad quality, bad anatomy, deformed body, deformed hands, deformed feet, deformed face, deformed clothing, deformed skin, bad skin, leggings, tights, stockings, pants, sleeves",

# pr = cProfile.Profile()
# pr.enable()
# start1 = time.time()

# image_res = pipe(
#                 prompt,
#                 # negative_prompt=negative_prompt,
#                 image=sm_image,
#                 control_image=sm_pose_image,
#                 control_mode=4,
#                 # padding_mask_crop=32,
#                 controlnet_conditioning_scale=controlnet_conditioning_scale,
#                 mask_image=sm_mask,
#                 height=height,
#                 width=width,
#                 strength=0.99,
#                 num_inference_steps=num_inference_steps,
#                 guidance_scale=guidance_scale,
#                 generator=generator,
#                 # prompt_embeds=embeds,
#                 # pooled_prompt_embeds=embeds,
#             ).images[0]
# end1 = time.time()
# with open('CN_benchmark/schnell_test.txt', 'w') as stream:
#     stats = Stats(pr,stream=stream)
#     stats.sort_stats('tottime').print_stats(50)

# pr.disable()
# image_res.save('CN_benchmark/schnell_test.png')
# image_res

In [ ]:
# pipe = FluxControlNetInpaintPipeline.from_pretrained("black-forest-labs/FLUX.1-schnell", controlnet=controlnet, torch_dtype=torch.bfloat16)

# pipe.enable_model_cpu_offload()
# pipe.text_encoder.to(torch.bfloat16)
# pipe.enable_sequential_cpu_offload()
# prompts=['a photo realistic image of a fashion model on a beach',
#         'a photo realistic image of a fashion model on a sailboat',
#         'a photo realistic image of a fashion model in a temple']
# for input_image in dir:
#     for j,prompt in enumerate(prompts):
#         sm_image, sm_pose_image, sm_mask= image_setup(input_image)
        
#         width,height=sm_image.size
        
#         num_inference_steps=30
#         guidance_scale=7
#         controlnet_conditioning_scale=1
#         seed=42
#         generator = torch.Generator(device="cuda").manual_seed(seed)
#         negative_prompt='ugly, bad quality, bad anatomy, deformed body, deformed hands, deformed feet, deformed face, deformed clothing, deformed skin, bad skin, leggings, tights, stockings, pants, sleeves'
#         i=os.path.basename(input_image).split('.')[0]
#         if j==0:
#             jj='beach'
#         elif j==1:
#             jj='boat'
#         elif j==2:
#             jj='temple'

#         pr = cProfile.Profile()
#         pr.enable()
#         start1 = time.time()
        

#         image_res = pipe(
#                 prompt,
#                 # negative_prompt=negative_prompt,
#                 image=sm_image,
#                 control_image=sm_pose_image,
#                 control_mode=4,
#                 # padding_mask_crop=32,
#                 controlnet_conditioning_scale=controlnet_conditioning_scale,
#                 mask_image=sm_mask,
#                 height=height,
#                 width=width,
#                 strength=0.99,
#                 num_inference_steps=num_inference_steps,
#                 guidance_scale=guidance_scale,
#                 generator=generator,
#                 # prompt_embeds=embeds,
#                 # pooled_prompt_embeds=embeds,
#             ).images[0]
#         end1 = time.time()
#         with open('schnell_benchmark/schnell_test'+str(i)+'_'+str(jj)+'.txt', 'w') as stream:
#             stats = Stats(pr,stream=stream)
#             stats.sort_stats('tottime').print_stats(50)
        
#         pr.disable()
#         # image_res.save('dev_benchmark/dev_test.png')
#         # image_res


        
#         image_res.save('schnell_benchmark/schnell_test_'+str(i)+'_'+str(jj)+'.png')
#         filename='schnell_benchmark/schnell_bench_'+str(i)+'_'+str(jj)+'.png'
#         showImagesHorizontally([sm_image,sm_mask,sm_pose_image,image_res], prompt,negative_prompt,'schnell',height, width, controlnet_conditioning_scale,num_inference_steps,guidance_scale,generator,output_path=filename)


#### OpenFLUX 700-1200mb gpumem 60-85% Volatile GPU-Util

In [ ]:
# import os
# import inspect
# ## change flux inpainting pipeline to allow negative-prompt in OpenFlux
# # PTH=(os.path.abspath(inspect.getfile(FluxControlNetInpaintPipeline)))
# PTH2=(os.path.abspath(inspect.getfile(FluxImg2ImgPipeline)))
# import shutil
# shutil.copy('LookBuilderPipeline/LookBuilderPipeline/image_models/openflux/cfg_pipeline_flux_controlnet_inpainting.py', PTH2)
# # PTH

In [ ]:
# import inspect
# PTH=(os.path.abspath(inspect.getfile(FluxControlNetInpaintPipeline)))
# PTH

In [ ]:
# from diffusers import FluxControlNetInpaintPipeline

# pipe = FluxControlNetInpaintPipeline.from_pretrained("ostris/OpenFLUX.1", controlnet=controlnet, torch_dtype=torch.bfloat16)

# pipe.enable_model_cpu_offload()
# pipe.text_encoder.to(torch.bfloat16)
# pipe.enable_sequential_cpu_offload()

# prompt='a photo realistic image of  black fashion model in atlanta georgia'
# negative_prompt="ugly, bad quality, bad anatomy, deformed body, deformed hands, deformed feet, deformed face, deformed clothing, deformed skin, bad skin, leggings, tights, stockings, pants, sleeves",


In [ ]:
# input_image=dir[2]
# sm_image, sm_pose_image, sm_mask= image_setup(input_image)

# height,width=sm_image.size
# generator = torch.Generator(device="cuda").manual_seed(42)


In [ ]:

# pr = cProfile.Profile()
# pr.enable()
# start1 = time.time()

# image_res = pipe(
#                 prompt,
#                 negative_prompt=negative_prompt,
#                 image=sm_image,
#                 control_image=sm_pose_image,
#                 control_mode=4,
#                 # padding_mask_crop=32,
#                 controlnet_conditioning_scale=0.9,
#                 mask_image=sm_mask,
#                 height=height,
#                 width=width,
#                 strength=0.99,
#                 num_inference_steps=20,
#                 guidance_scale=8,
#                 generator=generator,
#                 # prompt_embeds=embeds,
#                 # pooled_prompt_embeds=embeds,
#             ).images[0]
# end1 = time.time()
# with open('openflux_benchmark/openflux_test.txt', 'w') as stream:
#     stats = Stats(pr,stream=stream)
#     stats.sort_stats('tottime').print_stats(50)

# pr.disable()
# image_res.save('openflux_benchmark/openflux_test.png')
# image_res

In [ ]:
# pipe = FluxControlNetInpaintPipeline.from_pretrained("ostris/OpenFLUX.1", controlnet=controlnet, torch_dtype=torch.bfloat16)

# pipe.enable_model_cpu_offload()
# pipe.text_encoder.to(torch.bfloat16)
# pipe.enable_sequential_cpu_offload()
# prompts=['a photo realistic image of a fashion model on a beach',
#         'a photo realistic image of a fashion model on a sailboat',
#         'a photo realistic image of a fashion model in a temple']
# for input_image in dir:
#     for j,prompt in enumerate(prompts):
#         sm_image, sm_pose_image, sm_mask= image_setup(input_image)
#         i=os.path.basename(input_image).split('.')[0]
#         if j==0:
#             jj='beach'
#         elif j==1:
#             jj='boat'
#         elif j==2:
#             jj='temple'
#         width,height=sm_image.size
        
#         num_inference_steps=30
#         guidance_scale=7
#         controlnet_conditioning_scale=1
#         seed=42
#         generator = torch.Generator(device="cuda").manual_seed(seed)
#         negative_prompt='ugly, bad quality, bad anatomy, deformed body, deformed hands, deformed feet, deformed face, deformed clothing, deformed skin, bad skin, leggings, tights, stockings, pants, sleeves'
        
#         pr = cProfile.Profile()
#         pr.enable()
#         start1 = time.time()


#         image_res = pipe(
#                 prompt,
#                 negative_prompt=negative_prompt,
#                 image=sm_image,
#                 control_image=sm_pose_image,
#                 control_mode=4,
#                 # padding_mask_crop=32,
#                 controlnet_conditioning_scale=controlnet_conditioning_scale,
#                 mask_image=sm_mask,
#                 height=height,
#                 width=width,
#                 strength=0.99,
#                 num_inference_steps=num_inference_steps,
#                 guidance_scale=guidance_scale,
#                 generator=generator,
#                 # prompt_embeds=embeds,
#                 # pooled_prompt_embeds=embeds,
#             ).images[0]
#         end1 = time.time()
#         with open('openflux_benchmark/openflux_test'+str(i)+'_'+str(jj)+'.txt', 'w') as stream:
#             stats = Stats(pr,stream=stream)
#             stats.sort_stats('tottime').print_stats(50)
        
#         pr.disable()


#         image_res.save('openflux_benchmark/openflux_test_'+str(i)+'_'+str(jj)+'.png')
#         filename='openflux_benchmark/openflux_bench_'+str(i)+'_'+str(jj)+'.png'
#         showImagesHorizontally([sm_image,sm_mask,sm_pose_image,image_res], prompt,negative_prompt,'openflux',height, width, controlnet_conditioning_scale,num_inference_steps,guidance_scale,generator,output_path=filename)


#### QUANTIZED_OpenFLUX

In [ ]:
# from huggingface_hub import snapshot_download
# snapshot_download(repo_id="ostris/OpenFLUX.1",local_dir='flux-fp8')

In [24]:
from diffusers import FlowMatchEulerDiscreteScheduler, AutoencoderKL
from diffusers.models.transformers.transformer_flux import FluxTransformer2DModel
from diffusers.pipelines.flux.pipeline_flux import FluxPipeline
from transformers import CLIPTextModel, CLIPTokenizer,T5EncoderModel, T5TokenizerFast
dtype = torch.bfloat16
scheduler = FlowMatchEulerDiscreteScheduler.from_pretrained('flux-fp8', subfolder="scheduler")
text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14", torch_dtype=dtype)
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14", torch_dtype=dtype)
# text_encoder_2 = T5EncoderModel.from_pretrained('flux-fp8', subfolder="text_encoder_2", torch_dtype=dtype)
tokenizer_2 = T5TokenizerFast.from_pretrained('flux-fp8', subfolder="tokenizer_2", torch_dtype=dtype)
vae = AutoencoderKL.from_pretrained('flux-fp8', subfolder="vae", torch_dtype=dtype)
# transformer = FluxTransformer2DModel.from_pretrained('flux-fp8', subfolder="transformer", torch_dtype=dtype)

# Experimental: Try this to load in 4-bit for <16GB cards.
#
# from quanto import quantize, freeze, qint4, qint8, qfloat8
# from optimum.quanto import QuantizedModelForCausalLM, qfloat8, quantize,freeze
# pr = cProfile.Profile()
# pr.enable()
# start1 = time.time()
# quantize(transformer, weights=qfloat8)
# freeze(transformer)

# quantize(text_encoder_2, weights=qfloat8)
# freeze(text_encoder_2)
# end1 = time.time()
# with open('benchmark/Quanto_test.txt', 'w') as stream:
#     stats = Stats(pr,stream=stream)
#     stats.sort_stats('tottime').print_stats(50)

# pr.disable()


In [ ]:
# from safetensors.torch import save_file

# save_file(text_encoder_2.state_dict(), 'model.safetensors')

In [ ]:
# text_encoder_2.save_pretrained('qopenflux/text_encoder_2_qfloat8')

In [ ]:
# if glob.glob('qopenflux/text_encoder_2_qfloat8/*.safetensors'):
    # text_encoder_2 = T5EncoderModel.from_pretrained('qopenflux', subfolder="text_encoder_2_qfloat8", torch_dtype=dtype)
# else:
text_encoder_2 = T5EncoderModel.from_pretrained('flux-fp8', subfolder="text_encoder_2", torch_dtype=dtype)
quantize(text_encoder_2, weights=qfloat8)
freeze(text_encoder_2)
    # text_encoder_2.save_pretrained('qopenflux/text_encoder_2_qfloat8')

# # if glob.glob('qopenflux/transformer_qfloat8/*.safetensors'):
# try:
#     transformer = FluxTransformer2DModel.from_pretrained('qopenflux', subfolder="transformer_qfloat8", torch_dtype=dtype)
# # else:
# except:
transformer = FluxTransformer2DModel.from_pretrained('flux-fp8', subfolder="transformer", torch_dtype=dtype)
quantize(transformer, weights=qfloat8)
freeze(transformer)
    # transformer.save_pretrained('qopenflux/transformer_qfloat8')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# if glob.glob('qopenflux/text_encoder_2_int8/*.safetensors'):
#     text_encoder_2 = T5EncoderModel.from_pretrained('qopenflux', subfolder="text_encoder_2_int8", torch_dtype=dtype)
# else:
#     text_encoder_2 = T5EncoderModel.from_pretrained('flux-fp8', subfolder="text_encoder_2", torch_dtype=dtype)
#     quantize(text_encoder_2, weights=qint8)
#     freeze(text_encoder_2)
#     text_encoder_2.save_pretrained('qopenflux/text_encoder_2_int8')

# # if glob.glob('qopenflux/transformer/*.safetensors'):
# try:
#     transformer = FluxTransformer2DModel.from_pretrained('qopenflux', subfolder="transformer_int8", torch_dtype=dtype)
# # else:
# except:
#     transformer = FluxTransformer2DModel.from_pretrained('flux-fp8', subfolder="transformer", torch_dtype=dtype)
#     quantize(transformer, weights=qint8)
#     freeze(transformer)
#     transformer.save_pretrained('qopenflux/transformer_int8')

In [ ]:
# if glob.glob('qopenflux/text_encoder_2_int4/*.safetensors'):
#     text_encoder_2 = T5EncoderModel.from_pretrained('qopenflux', subfolder="text_encoder_2_int4", torch_dtype=dtype)
# else:
#     text_encoder_2 = T5EncoderModel.from_pretrained('flux-fp8', subfolder="text_encoder_2", torch_dtype=dtype)
#     quantize(text_encoder_2, weights=qint4)
#     freeze(text_encoder_2)
#     text_encoder_2.save_pretrained('qopenflux/text_encoder_2_int4')

# try:
#     transformer = FluxTransformer2DModel.from_pretrained('qopenflux', subfolder="text_encoder_2_int4", torch_dtype=dtype)
# except:
#     transformer = FluxTransformer2DModel.from_pretrained('flux-fp8', subfolder="transformer", torch_dtype=dtype)
#     quantize(transformer, weights=qint4)
#     freeze(transformer)
#     transformer.save_pretrained('qopenflux/transformer_int4')

In [ ]:
# text_encoder_2.save_pretrained('qopenflux/text_encoder_2_int4',strict=False)
# transformer.save_pretrained('qopenflux/transformer_int4',strict=False)

In [ ]:
# import os, inspect, shutil
# PTH=(os.path.abspath(inspect.getfile(FluxPipeline)))
# target='/'.join(PTH.split('/')[:-1])+'/pipeline_flux_controlnet_inpainting.py'
# source='LookBuilderPipeline/LookBuilderPipeline/image_models/openflux/cfg_pipeline_flux_controlnet_inpainting.py'
# shutil.copy(source, target)

#### QUANTIZED_OpenFLUX 22650mb gpumem 100% Volatile GPU-Util

In [ ]:
# from importlib import reload
# import diffusers
# reload(diffusers)
from diffusers import FluxControlNetInpaintPipeline #, FluxImg2ImgPipeline
# dir[12:13]

In [ ]:
# del pipe
# torch.cuda.empty_cache()

# from importlib import reload
# import optimum.quanto
# reload(optimum.quanto)

# from optimum.quanto import qfloat8,qint4,qint8, quantize,freeze

In [ ]:

pipe = FluxControlNetInpaintPipeline(
    controlnet=controlnet,
    scheduler=scheduler,
    text_encoder=text_encoder,
    tokenizer=tokenizer,
    text_encoder_2=None,
    tokenizer_2=tokenizer_2,
    vae=vae,
    transformer=None,
)

pipe.text_encoder_2 = text_encoder_2
pipe.transformer = transformer
pipe.enable_model_cpu_offload()

# # pipe.load_lora_weights('hugovntr/flux-schnell-realism', weight_name='schnell-realism_v1')
# pipe.load_lora_weights('hugovntr/flux-schnell-realism', weight_name='schnell-realism_v2.3.safetensors', adapter_name="winner")
# from diffusers import EulerAncestralDiscreteScheduler
# pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe3.scheduler.config)


In [ ]:
# pipe = FluxControlNetInpaintPipeline.from_pretrained("ostris/OpenFLUX.1", controlnet=controlnet, torch_dtype=torch.bfloat16)

# pipe.enable_model_cpu_offload()
# pipe.text_encoder.to(torch.bfloat16)
# pipe.enable_sequential_cpu_offload()

In [ ]:
from diffusers.utils import load_image  # For loading images
from transformers import pipeline  # For using pre-trained models
import numpy as np  # For numerical operations on arrays
from PIL import Image, ImageOps  # For image manipulation


# Initialize the segmentation model using a pre-trained model from Hugging Face
segmenter = pipeline(model="mattmdjaga/segformer_b2_clothes")


def full_mask(original_image,flux_image):
    seg_img = load_image(original_image)
    segments = segmenter(seg_img)
    segment_include = ["Upper-clothes", "Skirt", "Pants", "Dress", "Belt", "Bag", "Scarf","Left-shoe","Right-shoe"]
        
    seg_img = load_image(flux_image)
    segments = segmenter(seg_img)
    segment_include += ["Hat","Hair","Sunglasses","Face","Left-leg","Right-leg","Left-arm","Right-arm"]
        
        
    mask_list = [np.array(s['mask']) for s in segments if s['label'] not in segment_include]
    final_mask = np.array(mask_list[0])
    for mask in mask_list:
        current_mask = np.array(mask)
        final_mask = final_mask + current_mask
        
    final_array = final_mask.copy()
    final_mask = Image.fromarray(final_mask)
        # seg_img.putalpha(final_mask)
        
    mask = Image.new("L", final_mask.size)
    mask.paste(final_mask)
    mask = ImageOps.invert(mask)
    final_maskA = mask.point(lambda p: p > 128 and 255)
    # mask_blurred = pipeline.mask_processor.blur(final_maskA, blur_factor=20)
    return final_mask, final_maskA

In [ ]:
# prompt=[#'a photo realistic image of a fashion model at a party with curly brown hair and yelling something', \
#         # 'a photo realistic image of a fashion model with straight hair and hazel eyes and smiling showing white teeth at an ancient roman temple with many stone large columns and stone-paved streets at sunset']
#         'A very tall and slender model with longer-than-average arms and neck poses looking straight into the camera. 1. Model characteristics: - Height: Very tall and slender - Arms: Longer than average - Neck: Longer than average - Facial features: Very sharp, almost masculine but elegant - Expression: Very intriguing - Makeup: Completely nude look - Hair: Wet and slicked back with a wide-toothed comb 2. Lighting and effects: - Eyes: Shining intensely with the flash light - Skin: Completely glossy, highlighting the lines from the flash - Background: Light gray with a vignette - Shadows: Strongly defined by the flash, giving the impression the model is floating - Optical distortion: Continued - Setting: Roman++ Ampetheatre++ - Camera settings: Aperture at f/2.8']
# from compel import Compel, ReturnedEmbeddingsType
# compel = Compel(tokenizer=[pipe.tokenizer, pipe.tokenizer_2] , text_encoder=[pipe.text_encoder, pipe.text_encoder_2], returned_embeddings_type=ReturnedEmbeddingsType.PENULTIMATE_HIDDEN_STATES_NON_NORMALIZED, requires_pooled=[False, True])
# conditioning, pooled = compel(prompt)

In [ ]:
dir

In [ ]:
prompt=['a photo realistic image of a fashion model at a party with curly brown hair and yelling something']
        # 'a photo realistic image of a fashion model with straight hair and hazel eyes and smiling showing white teeth at an ancient roman temple with many stone large columns and stone-paved streets at sunset']
        # 'A very tall and slender model with longer-than-average arms and neck poses looking straight into the camera. 1. Model characteristics: - Height: Very tall and slender - Arms: Longer than average - Neck: Longer than average - Facial features: Very sharp, almost masculine but elegant - Expression: Very intriguing - Makeup: Completely nude look - Hair: Wet and slicked back with a wide-toothed comb 2. Lighting and effects: - Eyes: Shining intensely with the flash light - Skin: Completely glossy, highlighting the lines from the flash - Background: Light gray with a vignette - Shadows: Strongly defined by the flash, giving the impression the model is floating - Optical distortion: Continued - Camera settings: Aperture at f/2.8']
        # 'a photo realistic image of a fashion model with curly blonde hair and blue eyes and smirking on a busy city street corner under tall skyscrapers and neon lights at night']


num_inference_steps=40

for input_image in dir:
    for guidance_scale in [6,7,8,9]:
    # for j,prompt in enumerate(prompts):
        for controlnet_conditioning_scale in [1.0, 0.9, 0.7,0.5]:
            for strength in [0.3, 0.7, 0.9]:


                sm_image, sm_pose_image, sm_mask= image_setup(input_image)
                i=os.path.basename(input_image).split('.')[0]
                # if j==0:
                jj='temple'
                # elif j==1:
                    # jj='city'
                # elif j==2:
                #     jj='boat'
                width,height=sm_image.size
                
                # num_inference_steps=20
                seed=42
                
        
                generator = torch.Generator(device="cuda").manual_seed(seed)
                negative_prompt='ugly, bad quality, bad anatomy, deformed body, deformed hands, deformed feet, deformed face, deformed clothing, deformed skin, bad skin, leggings, tights, stockings, pants, sleeves'
                
                # pr = cProfile.Profile()
                # pr.enable()
                start1 = time.time()
        
        
                image_res = pipe(
                        prompt,
                        # prompt_embeds=conditioning,
                        # pooled_prompt_embeds=pooled,
                        negative_prompt=negative_prompt,
                        image=sm_image,
                        control_image=sm_pose_image,
                        control_mode=4,
                        padding_mask_crop=32,
                        controlnet_conditioning_scale=controlnet_conditioning_scale,
                        mask_image=sm_mask,
                        height=height,
                        width=width,
                        strength=strength,
                        num_inference_steps=num_inference_steps,
                        guidance_scale=guidance_scale,
                        generator=generator,
                        # prompt_embeds=embeds,
                        # pooled_prompt_embeds=embeds,
                    ).images[0]
                end1 = time.time()
                tt=end1-start1
                # with open('qopenflux_benchmark/qopenflux_test'+str(i)+'_'+str(jj)+'.txt', 'w') as stream:
                #     stats = Stats(pr,stream=stream)
                #     stats.sort_stats('tottime').print_stats(50)
                
                # pr.disable()
        
        
                image_res.save('qopenflux_benchmark/qopenflux_test_'+str(i)+'_'+str(jj)+'.png')
                filename='qopenflux_benchmark/qopenflux_bench_guid_'+str(i)+'_'+str(jj)+str(guidance_scale)+'_cond_'+str(controlnet_conditioning_scale)+'_strength_'+str(strength)+'_ints_'+str(num_inference_steps)+'.png'
                showImagesHorizontally([sm_image,sm_mask,sm_pose_image,image_res], prompt,negative_prompt,'openflux',tt,height, width, controlnet_conditioning_scale,num_inference_steps,guidance_scale,seed,strength,output_path=filename)


        # final_mask, final_maskA =full_mask(sm_image,image_res)
        # controlnet_conditioning_scale=0.9

        # image_res2 = pipe(
        #         prompt= 'a photo realistic image of a fashion model at an ancient roman temple with many stone large columns and stone-paved streets',
        #         negative_prompt=negative_prompt,
        #         image=image_res,
        #         control_image=sm_pose_image,
        #         control_mode=4,
        #         # padding_mask_crop=32,
        #         controlnet_conditioning_scale=controlnet_conditioning_scale,
        #         mask_image=final_mask,
        #         height=height,
        #         width=width,
        #         strength=strength,
        #         num_inference_steps=num_inference_steps,
        #         guidance_scale=guidance_scale,
        #         generator=generator,
        #         # prompt_embeds=embeds,
        #         # pooled_prompt_embeds=embeds,
        #     ).images[0]
        # end2 = time.time()
        # tt2=end2-start1

        # # image_res2.save('qopenflux_benchmark/qopenflux_test_'+str(i)+'_'+str(jj)+'.png')
        # filename='qopenflux_benchmark/out_qopenflux_bench_guid_'+str(i)+'_'+str(jj)+str(guidance_scale)+'_cond_'+str(controlnet_conditioning_scale)+'_strength_'+str(strength)+'_ints_'+str(num_inference_steps)+'.png'
        # showImagesHorizontally([sm_image,sm_mask,sm_pose_image,image_res2], prompt,negative_prompt,'openflux',tt,height, width, controlnet_conditioning_scale,num_inference_steps,guidance_scale,seed,strength,output_path=filename)



#### quanto-openflux NOCN

In [ ]:
from diffusers import FluxInpaintPipeline
pipe2 = FluxInpaintPipeline(
    scheduler=scheduler,
    text_encoder=text_encoder,
    tokenizer=tokenizer,
    text_encoder_2=None,
    tokenizer_2=tokenizer_2,
    vae=vae,
    transformer=None,
)


pipe2.text_encoder_2 = text_encoder_2
pipe2.transformer = transformer
pipe2.enable_model_cpu_offload()

# pipe.save_pretrained('qopenflux')
# transformer.save_pretrained('qopenflux/transformer/')
# text_encoder_2.save_pretrained('qopenflux/text_encoder_2/')


In [ ]:

# prompts=['a photo realistic image of a fashion model on a sailing boat with blonde curly hair and blue eyes and a crooked nose and pale skin', \
#         'a photo realistic image of a fashion model at an ancient roman temple  with curly brown hair and hazel eyes and smiling showing white teeth',  \
#         'a photo realistic image of a fashion model on a busy city street corner with curly brown hair and hazel eyes and smiling showing white teeth']

# input_image=dir[10]

# guidance_scale=6
# controlnet_conditioning_scale=0.7
# strength=0.9
# for j,prompt in enumerate(prompts):
#     # for controlnet_conditioning_scale in [0.7,0.9]: #,0.99]:
#     # for strength in [0.7,0.8,0.9,0.99]:
#     for num_inference_steps in [30]:

#         sm_image, sm_pose_image, sm_mask= image_setup(input_image)
#         i=os.path.basename(input_image).split('.')[0]
#         if j==0:
#             jj='boat'
#         elif j==1:
#             jj='temple'
#         elif j==2:
#             jj='city'
#         width,height=sm_image.size
        
#         # num_inference_steps=20
#         seed=42
        

#         generator = torch.Generator(device="cuda").manual_seed(seed)
#         negative_prompt='ugly, bad quality, bad anatomy, deformed body, deformed hands, deformed feet, deformed face, deformed clothing, deformed skin, bad skin, leggings, tights, stockings, pants, sleeves'
        
#         pr = cProfile.Profile()
#         pr.enable()
#         start1 = time.time()


#         image_res = pipe2(
#                 prompt,
#                 # negative_prompt=negative_prompt,
#                 image=sm_image,
#                 # control_image=sm_pose_image,
#                 # control_mode=4,
#                 padding_mask_crop=32,
#                 # controlnet_conditioning_scale=controlnet_conditioning_scale,
#                 mask_image=sm_mask,
#                 height=height,
#                 width=width,
#                 strength=strength,
#                 num_inference_steps=num_inference_steps,
#                 guidance_scale=guidance_scale,
#                 generator=generator,
#                 # prompt_embeds=embeds,
#                 # pooled_prompt_embeds=embeds,
#             ).images[0]
#         end1 = time.time()
#         tt=end1-start1
#         # with open('qopenflux_benchmark/qopenflux_test'+str(i)+'_'+str(jj)+'.txt', 'w') as stream:
#         #     stats = Stats(pr,stream=stream)
#         #     stats.sort_stats('tottime').print_stats(50)
        
#         pr.disable()


#         # image_res.save('qopenflux_benchmark/qopenflux_test_'+str(i)+'_'+str(jj)+'.png')
#         filename='qopenflux_benchmark/NOCN_qopenflux_bench_guid_'+str(i)+'_'+str(jj)+str(guidance_scale)+'_cond_'+str(controlnet_conditioning_scale)+'_strength_'+str(strength)+'_ints_'+str(num_inference_steps)+'.png'
#         showImagesHorizontally([sm_image,sm_mask,sm_pose_image,image_res], prompt,negative_prompt,'openflux',tt,height, width, controlnet_conditioning_scale,num_inference_steps,guidance_scale,seed,strength,output_path=filename)

## SDXL bench

In [ ]:
# import glob
# from random import shuffle
# # dir=glob.glob('test-ai/orig/*')
# # dir+=glob.glob('test-ai/jcrew_orig/*')
# # dir=glob.glob('test-ai/paulo_orig/*')
# # dir=glob.glob('test-ai/upscaled/*')
# dir=glob.glob('test-ai/more_paulo/*')

# shuffle(dir)

In [ ]:
# !pip install peft

In [ ]:
# from diffusers import StableDiffusionXLControlNetInpaintPipeline, ControlNetModel, DDIMScheduler

# controlnet_model3 = ControlNetModel.from_pretrained("xinsir/controlnet-union-sdxl-1.0", torch_dtype=torch.float16, use_safetensors=True,)


# pipe3 = StableDiffusionXLControlNetInpaintPipeline.from_pretrained(
#     # "stabilityai/stable-diffusion-xl-base-1.0", controlnet=controlnet_model, 
# #     "RunDiffusion/Juggernaut-XL-v9", 
#         "SG161222/RealVisXL_V5.0_Lightning",

# # pipe3 = StableDiffusionXLControlNetInpaintPipeline.from_single_file(
#     # "https://huggingface.co/RunDiffusion/Juggernaut-XL-v9/blob/main/Juggernaut-XL_v9_RunDiffusionPhoto_v2.safetensors", 
#     controlnet=controlnet_model3,
#     # vae=vae,
#     torch_dtype=torch.float16,
#     # scheduler=ddim_scheduler,
#     # scheduler=eulera_scheduler,
# )
# # model_path = "TheLastBen/William_Eggleston_Style_SDXL"


# # pipe = pipe.to(device)
# pipe3.text_encoder.to(torch.float16)
# pipe3.controlnet.to(torch.float16)
# # pipe3.unet.load_attn_procs(model_path, use_safetensors=True)
# # pipe3.load_lora_weights("TheLastBen/William_Eggleston_Style_SDXL",weight_name='wegg.safetensors', adapter_name="wegg")

# # pipe3.load_lora_weights('ntc-ai/SDXL-LoRA-slider.stylish-photoshoot', weight_name='stylish photoshoot.safetensors', adapter_name="style")
# # pipe3.load_lora_weights('ntc-ai/SDXL-LoRA-slider.Product-Photo', weight_name='Product Photo.safetensors', adapter_name="style")

# pipe3.to("cuda")

# # pipe3.load_lora_weights('ntc-ai/SDXL-LoRA-slider.extremely-detailed', weight_name='extremely detailed.safetensors', adapter_name="extremely detailed")

# # # Activate the LoRA
# # pipe3.set_adapters(["extremely detailed"], adapter_weights=[3.0])

# # pipe3.load_lora_weights('ntc-ai/SDXL-LoRA-slider.winner', weight_name='winner.safetensors', adapter_name="winner")

# # Activate the LoRA
# # pipe3.set_adapters(["winner"], adapter_weights=[2.0])
# # pipe.enable_model_cpu_offload()
# # pipe3.set_adapters(["style"], adapter_weights=[3.0])
# from diffusers import EulerAncestralDiscreteScheduler

# pipe3.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe3.scheduler.config)


In [ ]:
# prompt=[#'a photo realistic image of a fashion model at a party with curly brown hair and yelling something', \
#         # 'a photo realistic image of a fashion model with straight hair and hazel eyes and smiling showing white teeth at an ancient roman temple with many stone large columns and stone-paved streets at sunset']
#         'A very tall and slender model with longer-than-average arms and neck poses looking straight into the camera. 1. Model characteristics: - Height: tall and slender - Arms: Longer than average - Neck: Longer than average - Facial features: Very sharp, elegant - Expression: Very intriguing - Makeup: Completely nude look - Hair: Wet and slicked back with a wide-toothed comb 2. Lighting and effects: - Eyes: Shining intensely with the flash light - Skin: Completely glossy, highlighting the lines from the flash - Background: Roman Ampetheatre++ - Shadows: Strongly defined by the flash, giving the impression the model is floating - Optical distortion: Continued - Camera settings: Aperture at f/2.8']
       

In [ ]:
# from compel import Compel, ReturnedEmbeddingsType
# compel = Compel(tokenizer=[pipe3.tokenizer, pipe3.tokenizer_2] , text_encoder=[pipe3.text_encoder, pipe3.text_encoder_2], returned_embeddings_type=ReturnedEmbeddingsType.PENULTIMATE_HIDDEN_STATES_NON_NORMALIZED, requires_pooled=[False, True])
# conditioning, pooled = compel(prompt)

In [ ]:
# # prompt=[#'a photo realistic image of a fashion model at a party with curly brown hair and yelling something', \
# #         # 'a photo realistic image of a fashion model with straight hair and hazel eyes and smiling showing white teeth at an ancient roman temple with many stone large columns and stone-paved streets at sunset']
# #         'A very tall and slender model with longer-than-average arms and neck poses looking straight into the camera. 1. Model characteristics: - Height: Very tall and slender - Arms: Longer than average - Neck: Longer than average - Facial features: Very sharp, almost masculine but elegant - Expression: Very intriguing - Makeup: Completely nude look - Hair: Wet and slicked back with a wide-toothed comb 2. Lighting and effects: - Eyes: Shining intensely with the flash light - Skin: Completely glossy, highlighting the lines from the flash - Background: Light gray with a vignette - Shadows: Strongly defined by the flash, giving the impression the model is floating - Optical distortion: Continued - Camera settings: Aperture at f/2.8']

# # 'a photo realistic image of a fashion model with curly blonde hair and blue eyes and smirking on a busy city street corner under tall skyscrapers and neon lights at night']


# num_inference_steps=200
# for strength in [0.3]:
#     for guidance_scale in [10]:
#     # for j,prompt in enumerate(prompts):
#         for controlnet_conditioning_scale in [1.0]:

#             # for input_image in dir:
#             for input_image in dir[2:3]:


#                 sm_image, sm_pose_image, sm_mask= image_setup(input_image)
#                 i=os.path.basename(input_image).split('.')[0]
#                 # if j==0:
#                 jj='temple'
#                 # elif j==1:
#                     # jj='city'
#                 # elif j==2:
#                 #     jj='boat'
#                 width,height=sm_image.size
                
#                 # num_inference_steps=20
#                 seed=42
                
        
#                 generator = torch.Generator(device="cuda").manual_seed(seed)
#                 negative_prompt='ugly, bad quality, bad anatomy, deformed body, deformed hands, deformed feet, deformed face, deformed clothing, deformed skin, bad skin, leggings, tights, stockings, pants, sleeves'
                
#                 # pr = cProfile.Profile()
#                 # pr.enable()
#                 start1 = time.time()
        
        
#                 image_res = pipe3(
#                         # prompt,
#                         prompt_embeds=conditioning,
#                         pooled_prompt_embeds=pooled,
#                         negative_prompt=negative_prompt,
#                         image=sm_image,
#                         control_image=sm_pose_image,
#                         # control_mode=4,
#                         padding_mask_crop=32,
#                         controlnet_conditioning_scale=controlnet_conditioning_scale,
#                         mask_image=sm_mask,
#                         # height=height,
#                         # width=width,
#                         strength=strength,
#                         num_inference_steps=num_inference_steps,
#                         guidance_scale=guidance_scale,
#                         generator=generator,
#                         # prompt_embeds=embeds,
#                         # pooled_prompt_embeds=embeds,
#                     ).images[0]
#                 end1 = time.time()
#                 tt=end1-start1
#                 # with open('qopenflux_benchmark/qopenflux_test'+str(i)+'_'+str(jj)+'.txt', 'w') as stream:
#                 #     stats = Stats(pr,stream=stream)
#                 #     stats.sort_stats('tottime').print_stats(50)
                
#                 # pr.disable()
        
        
#                 image_res.save('sdxl_benchmark/more_paulo/sdxl_test_'+str(i)+'_'+str(jj)+'.png')
#                 filename='sdxl_benchmark/more_paulo/'+str(i)+'_'+str(jj)+'_guid_'+str(guidance_scale)+'_cond_'+str(controlnet_conditioning_scale)+'_strength_'+str(strength)+'_ints_'+str(num_inference_steps)+'.png'
#                 showImagesHorizontally([sm_image,sm_mask,sm_pose_image,image_res], prompt,negative_prompt,'openflux',tt,height, width, controlnet_conditioning_scale,num_inference_steps,guidance_scale,seed,strength,output_path=filename)

#                 torch.cuda.empty_cache()
            
#         # final_mask, final_maskA =full_mask(sm_image,image_res)
#         # controlnet_conditioning_scale=0.9

#         # image_res2 = pipe(
#         #         prompt= 'a photo realistic image of a fashion model at an ancient roman temple with many stone large columns and stone-paved streets',
#         #         negative_prompt=negative_prompt,
#         #         image=image_res,
#         #         control_image=sm_pose_image,
#         #         control_mode=4,
#         #         # padding_mask_crop=32,
#         #         controlnet_conditioning_scale=controlnet_conditioning_scale,
#         #         mask_image=final_mask,
#         #         height=height,
#         #         width=width,
#         #         strength=strength,
#         #         num_inference_steps=num_inference_steps,
#         #         guidance_scale=guidance_scale,
#         #         generator=generator,
#         #         # prompt_embeds=embeds,
#         #         # pooled_prompt_embeds=embeds,
#         #     ).images[0]
#         # end2 = time.time()
#         # tt2=end2-start1

#         # # image_res2.save('qopenflux_benchmark/qopenflux_test_'+str(i)+'_'+str(jj)+'.png')
#         # filename='qopenflux_benchmark/out_qopenflux_bench_guid_'+str(i)+'_'+str(jj)+str(guidance_scale)+'_cond_'+str(controlnet_conditioning_scale)+'_strength_'+str(strength)+'_ints_'+str(num_inference_steps)+'.png'
#         # showImagesHorizontally([sm_image,sm_mask,sm_pose_image,image_res2], prompt,negative_prompt,'openflux',tt,height, width, controlnet_conditioning_scale,num_inference_steps,guidance_scale,seed,strength,output_path=filename)

